In [1]:
!pip install evaluate
!pip install bitsandbytes

In [2]:
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    pipeline
)
import torch
import pandas as pd
from rich.console import Console
from rich.table import Table
import re
from tqdm.auto import tqdm
import evaluate

In [3]:
!pip install --upgrade datasets

In [4]:
from datasets import load_dataset
dataset = load_dataset("SajjadAyoubi/persian_qa")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

persian_qa.py: 0.00B [00:00, ?B/s]

0000.parquet:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/165k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/930 [00:00<?, ? examples/s]

In [5]:
console = Console()

In [6]:
table = Table(title="Original Dataset Analysis")
table.add_column("Split", style="magenta")
table.add_column("Total Examples", justify="right", style="green")

In [7]:
for split_name in dataset.keys():
    split_data = dataset[split_name]
    total_count = len(split_data)

    table.add_row(
        split_name,
        str(total_count),
    )

console.print(table)

   Original Dataset Analysis   
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Split      ┃ Total Examples ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ train      │           9008 │
│ validation │            930 │
└────────────┴────────────────┘

In [8]:
model1 = pipeline("question-answering", model="pedramyazdipoor/persian_xlm_roberta_large" , device=0)

model2 = pipeline("question-answering", model="pedramyazdipoor/parsbert_question_answering_PQuAD" , device=0)

config.json:   0%|          | 0.00/821 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/621 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

Some weights of the model checkpoint at pedramyazdipoor/parsbert_question_answering_PQuAD were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at pedramyazdipoor/parsbert_question_answering_PQuAD and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be 

tokenizer_config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [9]:
squad_metric = evaluate.load("squad_v2")

In [10]:
def cleaner(text):
    return re.sub('\u200c', " ", text).strip()

def evaluate_model(qa_pipeline, dataset_split, batch_size=32):
    """
    Evaluates a question-answering pipeline on a given dataset split using batched inference.

    Args:
        qa_pipeline: A Hugging Face question-answering pipeline.
        dataset_split: The dataset split to evaluate on.
        batch_size (int): The number of examples to process at once. Adjust based on GPU memory.

    Returns:
        A dictionary with the evaluation results (exact_match, f1).
    """
    print(f"Evaluating model: {qa_pipeline.model.name_or_path} using a batch size of {batch_size}...")
    questions = list(dataset_split["question"])
    contexts = list(dataset_split["context"])


    all_predictions = qa_pipeline(question=questions, context=contexts, batch_size=batch_size, truncation=True)

    predictions = []
    references = []

    for example, prediction_dict in zip(dataset_split, all_predictions):
        id_str = str(example["id"])
        formatted_prediction = {
            "id": id_str,
            "prediction_text": cleaner(prediction_dict["answer"]),
            "no_answer_probability": 0.0
        }

        formatted_reference = {
            "id": id_str,
            "answers": {'answer_start': example['answers']['answer_start'] ,
                        'text': map(cleaner , example["answers"]['text'])}
        }

        predictions.append(formatted_prediction)
        references.append(formatted_reference)

    results = squad_metric.compute(predictions=predictions, references=references)
    return results

In [11]:
results_model2 = evaluate_model(model2, dataset['validation'])
print("\nResults for pedramyazdipoor/parsbert_question_answering_PQuAD:")
print(results_model2)

print("\n" + "="*50 + "\n")

results_model1 = evaluate_model(model1, dataset['validation'])
print("\nResults for pedramyazdipoor/persian_xlm_roberta_large:")
print(results_model1)

Evaluating model: pedramyazdipoor/parsbert_question_answering_PQuAD using a batch size of 32...

Results for pedramyazdipoor/parsbert_question_answering_PQuAD:
{'exact': 0.0, 'f1': 5.414132878374805, 'total': 930, 'HasAns_exact': 0.0, 'HasAns_f1': 7.734475540535436, 'HasAns_total': 651, 'NoAns_exact': 0.0, 'NoAns_f1': 0.0, 'NoAns_total': 279, 'best_exact': 30.0, 'best_exact_thresh': 0.0, 'best_f1': 30.01955034213099, 'best_f1_thresh': 0.0}


Evaluating model: pedramyazdipoor/persian_xlm_roberta_large using a batch size of 32...

Results for pedramyazdipoor/persian_xlm_roberta_large:
{'exact': 36.88172043010753, 'f1': 54.83229050853049, 'total': 930, 'HasAns_exact': 52.68817204301075, 'HasAns_f1': 78.331843583615, 'HasAns_total': 651, 'NoAns_exact': 0.0, 'NoAns_f1': 0.0, 'NoAns_total': 279, 'best_exact': 37.74193548387097, 'best_exact_thresh': 0.0, 'best_f1': 55.239042791642326, 'best_f1_thresh': 0.0}


In [12]:
max_length = 384
doc_stride = 128

In [13]:
def preprocess_function(examples , tokenizer):
    questions = [q.strip() for q in examples["question"]]


    tokenized_examples = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )


    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")


    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]


        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:

            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [14]:
def fine_tune_model(model_name , model_checkpoint , args):

    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    print(f"Applying preprocessing function to {model_name}...")
    tokenized_datasets = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset["train"].column_names,
        fn_kwargs={'tokenizer': tokenizer}
    )

    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
    )
    print("Starting the Training...")
    trainer.train()

    print("Training finished. Saving model...")
    trainer.save_model(f"./{model_name}/final")

In [15]:
MODEL1_CHECKPOINT = "pedramyazdipoor/persian_xlm_roberta_large"
MODEL1_NAME = "persian-xlm-roberta-large-finetuned-persianqa"

#for roberta
args1 = TrainingArguments(
        output_dir=MODEL1_NAME,
        eval_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=8,
        num_train_epochs=3,
        weight_decay=0.01,
        disable_tqdm=False,
        report_to="none",
        save_strategy = "epoch",
        warmup_ratio = 0.1,
        fp16=True,
        optim="adamw_8bit"

    )

MODEL2_CHECKPOINT = "pedramyazdipoor/parsbert_question_answering_PQuAD"
MODEL2_NAME = "parsbert-question-answering-PQuAD-finetuned-persianqa"

#for parsbert
args2 = TrainingArguments(
        output_dir=MODEL1_NAME,
        eval_strategy="epoch",
        learning_rate=3e-5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=8,
        num_train_epochs=3,
        weight_decay=0.01,
        disable_tqdm=False,
        report_to="none",
        save_strategy = "epoch",
        warmup_ratio = 0.1,

    )

In [16]:
fine_tune_model(model_name = MODEL1_NAME ,
                model_checkpoint = MODEL1_CHECKPOINT ,
                args = args1)

Applying preprocessing function to persian-xlm-roberta-large-finetuned-persianqa...


Map:   0%|          | 0/9008 [00:00<?, ? examples/s]

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

/tmp/ipython-input-14-1926114161.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting the Training...


Epoch,Training Loss,Validation Loss
1,1.544100,1.154170
2,1.005300,1.217602
3,0.550700,1.432584


Training finished. Saving model...


In [17]:
fine_tune_model(model_name = MODEL2_NAME ,
                model_checkpoint = MODEL2_CHECKPOINT ,
                args = args2)

Applying preprocessing function to parsbert-question-answering-PQuAD-finetuned-persianqa...


Map:   0%|          | 0/9008 [00:00<?, ? examples/s]

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

Some weights of the model checkpoint at pedramyazdipoor/parsbert_question_answering_PQuAD were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at pedramyazdipoor/parsbert_question_answering_PQuAD and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be 

Starting the Training...


Epoch,Training Loss,Validation Loss
1,2.116400,1.556500
2,1.285200,1.538016
3,0.658900,1.862442


Training finished. Saving model...


In [18]:
FINETUNED_MODEL_1_PATH = "./persian-xlm-roberta-large-finetuned-persianqa/final"
FINETUNED_MODEL_2_PATH = "./parsbert-question-answering-PQuAD-finetuned-persianqa/final"

pipe_finetuned_1 = pipeline("question-answering", model=FINETUNED_MODEL_1_PATH, device=0)
pipe_finetuned_2 = pipeline("question-answering", model=FINETUNED_MODEL_2_PATH, device=0)

results_finetuned_1 = evaluate_model(pipe_finetuned_1, dataset['validation'])

results_finetuned_2 = evaluate_model(pipe_finetuned_2, dataset['validation'])


Device set to use cuda:0
Device set to use cuda:0


Evaluating model: ./persian-xlm-roberta-large-finetuned-persianqa/final using a batch size of 32...
Evaluating model: ./parsbert-question-answering-PQuAD-finetuned-persianqa/final using a batch size of 32...


In [20]:
console = Console()
table = Table(
    title="Project Results",
    show_header=True,
    header_style="bold magenta",
    show_lines=True
)

table.add_column("Model Name", style="cyan", width=35)
table.add_column("State", style="green")
table.add_column("Exact Match (%)", justify="right")
table.add_column("Answerable Exact Match (%)", justify="right")
table.add_column("UnAnswerable Exact Match (%)", justify="right")
table.add_column("F1-Score (%)", justify="right")
table.add_column("Answerable F1-Score (%)", justify="right")
table.add_column("UnAnswerable F1-Score (%)", justify="right")



f1_baseline_1 = results_model1['f1']
f1_finetuned_1 = results_finetuned_1['f1']


table.add_row(
    "roberta",
    "Baseline",
    f"{results_model1['exact']:.2f}",
    f"{results_model1['HasAns_exact']:.2f}",
    f"{results_model1['NoAns_exact']:.2f}",
    f"{results_model1['f1']:.2f}",
    f"{results_model1['HasAns_f1']:.2f}",
    f"{results_model1['NoAns_f1']:.2f}",
)
table.add_row(
    "roberta",
    "Fine-Tuned",
    f"{results_finetuned_1['exact']:.2f}",
    f"{results_finetuned_1['HasAns_exact']:.2f}",
    f"{results_finetuned_1['NoAns_exact']:.2f}",
    f"{results_finetuned_1['f1']:.2f}",
    f"{results_finetuned_1['HasAns_f1']:.2f}",
    f"{results_finetuned_1['NoAns_f1']:.2f}",
)


table.add_section()


table.add_row(
    "parsbert",
    "Baseline",
    f"{results_model2['exact']:.2f}",
    f"{results_model2['HasAns_exact']:.2f}",
    f"{results_model2['NoAns_exact']:.2f}",
    f"{results_model2['f1']:.2f}",
    f"{results_model2['HasAns_f1']:.2f}",
    f"{results_model2['NoAns_f1']:.2f}",
)
table.add_row(
    "parsbert",
    "Fine-Tuned",
    f"{results_finetuned_2['exact']:.2f}",
    f"{results_finetuned_2['HasAns_exact']:.2f}",
    f"{results_finetuned_2['NoAns_exact']:.2f}",
    f"{results_finetuned_2['f1']:.2f}",
    f"{results_finetuned_2['HasAns_f1']:.2f}",
    f"{results_finetuned_2['NoAns_f1']:.2f}",
)



console.print(table)

                                                  Project Results                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┓
┃                                     ┃          ┃          ┃ Answera… ┃ UnAnswe… ┃          ┃          ┃         ┃
┃                                     ┃          ┃    Exact ┃    Exact ┃    Exact ┃          ┃ Answera… ┃ UnAnsw… ┃
┃                                     ┃          ┃    Match ┃    Match ┃    Match ┃ F1-Score ┃ F1-Score ┃ F1-Sco… ┃
┃ Model Name                          ┃ State    ┃      (%) ┃      (%) ┃      (%) ┃      (%) ┃      (%) ┃     (%) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━┩
│ roberta                             │ Baseline │    36.88 │    52.69 │     0.00 │    54.83 │    78.33 │    0.00 │
├─────────────────────────────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ roberta                             │ Fine-Tu… │    36.02 │    51.46 │     0.00 │    55.06 │    78.66 │    0.00 │
├─────────────────────────────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ parsbert                            │ Baseline │     0.00 │     0.00 │     0.00 │     5.41 │     7.73 │    0.00 │
├─────────────────────────────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ parsbert                            │ Fine-Tu… │    33.76 │    48.23 │     0.00 │    52.38 │    74.83 │    0.00 │
└─────────────────────────────────────┴──────────┴──────────┴──────────┴──────────┴──────────┴──────────┴─────────┘

In [ ]:
def find_prediction_errors(pipeline, dataset_split):
    """
    Finds and collects incorrect predictions made by the model.
    """
    errors = []

    all_predictions = pipeline(
        question=list(dataset_split["question"]),
        context=list(dataset_split["context"]),
        batch_size=16
    )

    for i, example in enumerate(dataset_split):
        true_answers = example["answers"]["text"]
        predicted_answer = all_predictions[i]["answer"]

        is_correct = any(predicted_answer.strip().lower() == true_ans.strip().lower() for true_ans in true_answers)

        if not is_correct and true_answers:
            errors.append({
                "question": example["question"],
                "context": example["context"],
                "true_answers": true_answers,
                "predicted_answer": predicted_answer
            })

    return errors

roberta_errors = find_prediction_errors(pipe_finetuned_1, dataset['validation'])
parsbert_errors = find_prediction_errors(pipe_finetuned_2, dataset['validation'])
roberta_df = pd.DataFrame(roberta_errors)
parsbert_df = pd.DataFrame(parsbert_errors)
roberta_df.to_csv('roberta_errors.csv', index=False, encoding='utf-8-sig')
parsbert_df.to_csv('parsbert_errors.csv', index=False, encoding='utf-8-sig')